#### CIE4604 Simulation and Visualization
# Module 2 Satellite Orbits - Exercise 3

**Hans van der Marel, 19 November 2020**

In this exercise we show how to compute the look angles for an observer to the satellite of interest, and vice versa, from the satellite to a point on Earth (or in space).

## Import crsutil.py and tleplot.py Python modules

For the first time, download `CIE4604-M2-python.zip` from Brightspace and unzip this in your current working directory. This should give you two Python modules: `crsutil.py` and `tleplot.py`.

For this Jupyter notebook to work, the two Python modules should be in the same folder as this notebook. Import the modules using the following statements:

In [ ]:
import numpy as np
import crsutil as crs
import tleplot as tle

To get the docs for a specific function, use `help(functionname)`, or, to get the docs on all the functions at once do `help(module)`. To get help in a separate window type `module?` or `functionname?`. 


## Read TLE for Earth Resource Satellites

First read the Two Line Elements (TLE) for Earth Resource Satellite, that contain Radarsat-2 (amongst others), and save the 
results in a structure array *tleERS* with `tleread`

In [ ]:
tleERS = tle.tleread('resource-10-oct-2017.tle', verbose=0)

To suppress output during the reading we have added the option `verbose=0`
to the call of `tleread`.

## Test case (Compute satellite and observer position) 

Initialy, we will test the code for computing the look angles in this script first on one epoch. We select an epoch for which we know Radarsat-2 is in the proximity of Delft. How to compute the time a satellite is visibible, or at its closest point,
is something we will study in later exercises.

### Satellite position in ECI

We compute the satellite position in an Earth Centered Intertial frame

In [ ]:
t = tle.tledatenum('2017-9-28 6:01')

xsat, vsat = tle.tle2vec(tleERS, t, 'RADARSAT-2')

print('t',t)
print('xsat: ', xsat)
print('vsat: ', vsat)

At this time Radarsat-2 is on a descending track East of Delft. We can
verify this by calling `tleplot1` using a 20-minute window around this 
time.

In [ ]:
tle.tleplot1(tleERS,['2017-9-28 5:51', 20 ,.1],'RADARSAT-2',[ 52, 4.8,  0 ])

### Position of the observer in ECEF

The position of the observer (latitude, longitude and height) is given in ECEF using the input array *objcrd*.
We convert this to *lat* and *lon* in radians and ECEF coordinates.

In [ ]:
# Latitude, longitude and height of the observer

objcrd=[ 52, 4.8,  0 ]

# Position of the observer in ECEF (assume latitude and longitude are for spherical Earth)

Re = 6378136;              # [m]   radius of the Earth 

# The position of the observer (latitude, longitude and height) is given in ECEF
# using the input array objcrd
  
lat = objcrd[0]*np.pi/180  # convert latitude from degrees to radians
lon = objcrd[1]*np.pi/180  # convert longitude from degrees to radians
Rs = Re + objcrd[2]        # convert height to radius (from CoM) using mean Earth radius Re

# Position of the observer in ECEF (assume latitude and longitude are for spherical Earth)

xobjECEF = [Rs*np.cos(lat)*np.cos(lon),
            Rs*np.cos(lat)*np.sin(lon),
            Rs*np.sin(lat)]
vobjECEF = [0, 0, 0]

print('xobjECEF',xobjECEF)

### Conversion to common reference frame

For the computation of the zenith and azimuth angle both the observer
position and satellite position must both be in an ECI reference frame,
or both in ECEF. So, first we have to convert the observer position 
into the ECI reference frame.

The transformation from an ECEF to ECI is a simple rotation around the z-axis

1. the rotation angle is GMST (Greenwhich Mean Stellar Time)

2. the rotation around the z-axis can be implemented by replacing the
   longitude (in ECEF) by local stellar time (lst) in the ECI

The times given in *t* are in UTC, which is close to UT1 (max 0.9 s difference),
which is not important for a plotting application

Compute GMST from UT1, for the first epoch in *t*, using the function 
ut2gmst. The second output returned by ut2gmst is the rotational velocity
omegae of the Earth in rev/day.
```
gst0, omegae = ut2gmst(t[0])
```
Compute local stellar time (in radians) from the longitude, GMST at the initial
epoch and the rotational velocity of the Earth (times elapsed time). Note that
lst is an array, while lon is a scalar)
```
lst = lon + gst0 +2 * np.pi * omegae * (t-[0])
```
Finally, compute position and velocity of the observer in ECI using lst (position and 
velocity in an ECI change all the time, unlike in a ECEF).

The full algorithm is

In [ ]:
# Compute GMST from UT1, for the first epoch in t, using the Matlab function 
# ut2gmst. The second output returned by ut2gmst is the rotational velocity
# omegae of the Earth in rev/day

gst0, omegae = crs.ut2gmst(t[0])

Me = 7.2921151467e-5       # [rad/sec] rotational velocity of the Earth -> linked to omegae

# Compute local stellar time (in radians) from the longitude, GMST at the initial
# epoch and the rotational velocity of the Earth (times elapsed time). Note that
# lst is an array, while lon is a scalar)

lst = lon + gst0 + 2*np.pi*omegae*(t-t[0])

# Compute position and velocity of the observer in ECI using lst (position and 
# velocity in an ECI change all the time, unlike in a ECEF)

nepoch = t.shape[0]

xobj = np.zeros((nepoch, 3))   # pre-allocate memory, makes it run faster
vobj = np.zeros((nepoch, 3))

xobj[:, 0] = Rs * np.cos(lat) *np.cos(lst)
xobj[:, 1] = Rs * np.cos(lat) * np.sin(lst)
xobj[:, 2] = Rs * np.sin(lat)

vobj[:, 0] = -Rs * np.cos(lat) * Me * np.sin(lst)
vobj[:, 1] =  Rs * np.cos(lat) * Me * np.cos(lst)

print('xobj',xobj)
print('vobj',vobj)

### Relative position, velocity, range and range-rate

Next we compute the position and velocity vectors with respect to the
observer, and then the range and range rates between the observer and satellite.

In [ ]:
# Relative position and velocity vectors from object to satellite

xobj2sat = xsat - xobj
vobj2sat = vsat - vobj

# range and range rates between object and satellite 

robj2sat = np.sqrt(np.sum(xobj2sat**2, axis=1))
rrobj2sat = np.sum(vobj2sat * xobj2sat/robj2sat[:, np.newaxis], axis=1)

print('xobj2sat',xobj2sat)
print('vobj2sat',vobj2sat)
print('robj2sat',robj2sat)
print('rrobj2sat',rrobj2sat)

Note that the range rate *rrobj2sat* is not the same as the relative velocity
`np.sqrt(np.sum(vobj2sat**2, axis=1))`, these are different things.

## Zenith angle and azimuth of satellite

> Make a small code snippet to compute the zenith angle and azimuth of a satellite as seen 
from an observer on Earth (using observer position in ECEF). 

### Look angles

The various angles are shown in the plot below. 

**P** is the position of the observer, object or point on Earth, **S** is the satellite and **S'** is the sub-satellite point on the Earth surface. $\mathbf{x}_P$ and $\mathbf{x}_S$ are the position vectors from the center of the Earth **O** to the point on Earth **P** and satellite **S**. These are vectors with three Cartesian coordinates (*X, Y, Z*)

![](satellite_look_angles_3D_small.png)

The *zenith angle* from **P** to **S** is called $z$, the *azimuth angle* with respect to the meridian through **P**, and in the direction to **S'**, is called $\alpha_{PS}$. 

The azimuth angle for the satellite is *not* a simple *180* deg difference due to the curvature of the Earth. The azimuth angle for the satellite **S** to the point **P** on Earth is the angle $\alpha_{SP}$, this is the angle in **S'** to **P** with respect to the meridian through **S'**. The "equivalent" of the zenith angle for the satellite is the so-called nadir (looking) angle, which is called $\eta$. This angle is with respect to the nadir looking vector, $-\mathbf{x_S}$.

In the figure on the right also the direction of travel of the satellite with the velocity vector $\mathbf{v}_S$ is plotted. The heading of the satellite is the angle the projection of the vector $\mathbf{v}_S$ on the Earth surface makes with the local meridian through point **S'**, this angle is called $\alpha_{vsat}$. The angle $l$ gives the direction of the point $P$ with respect to the satellite heading, i.e. $l = \alpha_{SP} - \alpha_{vsat}$. 

### Algorithm derivation

The algorithm which we are about to use is based on the transformation 
between 3D cartesian coordinate differences in the ECI (or ECEF) frame 
and a local North, East, Up (NEU) system.

The relation between the differential coordinates $(dX,dY,dZ)$  and  $(dN,dE,dU)$ is,
 
$\left( \begin{array}{c} dX \\ dY \\dZ \end{array}\right) =
 \left( \begin{array}{ccc} - \sin \varphi \cos \lambda &  - \sin \lambda & \cos \varphi \cos \lambda \\
                                         - \sin \varphi \sin \lambda &   \cos \lambda & \cos \varphi \sin \lambda \\
                                            \cos \varphi &  &  \sin \varphi  \end{array} \right)
 \left( \begin{array}{c} dN \\ dE  \\ dU  \end{array} \right) $

The inverse relation is

$\left( \begin{array}{c} dN \\ dE \\dU \end{array}\right) =
 \left( \begin{array}{ccc} - \sin \varphi \cos \lambda & - \sin \varphi \sin \lambda &  \cos \varphi  \\
                                        - \sin \lambda &                \cos \lambda &  0 \\
                             \cos \varphi \cos \lambda &   \cos \varphi \sin \lambda &  \sin \varphi  \end{array} \right)
 \left( \begin{array}{c} dX \\ dY  \\ dZ  \end{array} \right) $

see Eq. 4.15 of the Reference System reader.

In the third row of the rotation matrix we recognise the transpose of the normal
vector $\bar{\mathbf n}$, with

$ \bar{\mathbf n}^T =  \left( \begin{array}{ccc} n_1 & n_2 & n_3 \end{array} \right) = \left( \begin{array}{ccc} \cos \varphi \cos \lambda &   \cos \varphi \sin \lambda &  \sin \varphi  \end{array} \right)$

Using this, the transformation can be re-written as

$\left( \begin{array}{c} dN \\ dE \\dU \end{array}\right) =
 \left( \begin{array}{ccc} - n_1 n_3 / \cos \varphi    & - n_2 n_3 / \cos \varphi  &  \cos \varphi  \\
                           - n_2 \cos \varphi          &   n_1 / \cos \varphi      &  0 \\
                             n_1                       &   n_2                     &  n_3    \end{array} \right)
 \left( \begin{array}{c} dX \\ dY  \\ dZ  \end{array} \right) $

Now, if we define two new variables, representing the inner product of 
the normal vector $\bar{\mathbf n}$ with $(dX,dY,dZ)$ and the cosine of
the latitude,

$\mbox{ip} =  n_1 * dX + n_2 * dY +  n_3 * dZ $

$\mbox{cphi} = \cos \varphi = \sqrt{ 1 -n_3^2}$

we obtain after some re-writing

$dN = ( dZ - \mbox{ip} * n_3 ) / \mbox{cphi} $
 
$dE = ( - n_2 * dX +  n_1 * dY ) / \mbox{cphi} $

$dU = \mbox{ip} $

For the distance $s$, zenith angle $z$  and azimuth $\alpha$ we find

$s =\sqrt{dX^2 + dY^2 + dZ^2}$

$z = \mbox{acos}( \mbox{ip} / s)$

$\alpha = \mbox{atan2}( dE , dN )$

As you can see, the final equations are quite elegant and efficient, and easy to implement.

### Algorithm

The azimuth and elevation are computed from the unit direction vector
*ers* from the observer to the satellite, and normal vector *n0* for the
observer, which are computed first. The zenith angle and azimuth angle
are computed next.

In [ ]:
# normal vector (vertical) and unit direction vector to satellite from observer

robj = np.sqrt(np.sum(xobj**2, axis=1))    # range to the object (observer)
n0 = xobj / robj[:, np.newaxis]            # normal vector from object (observer)
ers = xobj2sat / robj2sat[:, np.newaxis]   # unit direction vector from observer to satellite

# zenith angle and azimuth of satellite (as seen from object wrt to radial direction)

ip = np.sum(n0 * ers, axis=1)
zenith = np.arccos(ip)
azi = np.arctan2(-n0[:, 1]*ers[:, 0] + n0[:, 0]*ers[:, 1], ip*-1*n0[:, 2] + ers[:, 2])
azi += 2*np.pi
azi %= 2*np.pi

# print result

print('Zenith angle (rad->deg)',zenith*180/np.pi)
print('Azimuth angle (rad->deg)',azi*180/np.pi)

> Add a test to determine whether or not a satellite is visible from the observer, assuming a certain cut-off elevation angle.  

First, compute the elevation angle, in degrees, from the zenith angle, then test is the elevation angle is above the cutoff limit.



In [ ]:
# Elevation angle and satellite visibility (if elevation angle > 0))
    
elevation = np.pi/2 - zenith

cutoff = 0                       # cutoff angle in degrees
visible = elevation > np.radians(cutoff)

print(visible)

Here we used a cut-off elevation angle of zero degrees, which assumes a
clear horizon. A more realistic choice could have been ten degrees.

The computation above is done in an ECI reference frame, but it
is also posible to do this in an ECEF reference frame. For this you have
to replace *xsat* by *xsate*, and *xobj* by *xobjECEF*.

The velocities have not be used in this part of the computation, but we
need those later on to compute the look-angle.

To rewrite this code into a function is straighforward and not shown
here.


### Nadir angle and azimuth angle from satellite

> Make a small code snippet to compute the nadir and azimuth angle from the satellite 
to a point on Earth (with coordinates in ECEF).
 
The computation of the nadir angle and azimuth angle from the satellite
to the observer is similar to the computation of zenith angle and
azimuth in the previous section. You only have to replace *xobj* by
*xsat*, and revert the direction of *ers*.


In [ ]:
# normal vector (vertical) and unit direction vector to satellite from observer

rsat = np.sqrt(np.sum(xsat**2, axis=1))    # range to the satellite
n0sat = xsat / rsat[:, np.newaxis]         # normal vector from satellite
ers = xobj2sat / robj2sat[:, np.newaxis]   # unit direction vector from observer to satellite

# zenith angle and azimuth of satellite (as seen from object wrt to radial direction)

ipsat = np.sum(n0sat * ers, axis=1)
nadir = np.arccos(ipsat)
azisat = np.arctan2( n0sat[:, 1]*ers[:, 0] - n0sat[:, 0]*ers[:, 1], -ipsat*-1*n0sat[:, 2] - ers[:, 2])  # note sign difference in ers
azisat += 2*np.pi
azisat %= 2*np.pi

# print result

print('Nadig angle (rad->deg)',nadir*180/np.pi)
print('Azimuth angle (rad->deg)',azisat*180/np.pi)

# print difference with angles that originate at the observer 
print('Zenith and Nadir angle difference',(zenith-nadir)*180/np.pi)
print('Azimuth angle difference',(azi-azisat)*180/np.pi)

> Why is the nadir angle not equal to the zenith angle, and why is the difference between azimuth angles from the two view points not 180 degrees?

This is because we are working on a sphere and because the angles are
defined with respect to a local normal and north direction. 

To rewrite this code into a function is straighforward and not shown
here.

## Look angle from satellite

Compute the look angle from the satellite to a point on Earth. The look angle 
is the difference between the azimuth computed in the previous section) and the azimuth angle 
of the velocity vector (direction of travel).    

The azimuth of the velocity vector is

In [ ]:
velsat = np.sqrt(np.sum(vsat**2, axis=1))    # velocity

evsat = vsat / velsat[:, np.newaxis]         # unit direction vector from velocitu

ipvel = np.sum(n0sat * evsat, axis=1)
azivel = np.arctan2( -n0sat[:, 1]*evsat[:, 0] + n0sat[:, 0]*evsat[:, 1], ipvel*-1*n0sat[:, 2] + evsat[:, 2])  # note sign difference in ers
azivel += 2*np.pi
azivel %= 2*np.pi

print('Azimuth angle to observer',azisat*180/np.pi)
print('Azimuth angle to velocity vector',azivel*180/np.pi)


The look angle is the difference of the two azimuth angles:

In [ ]:
print('Look angle',(azisat-azivel)*180/np.pi)

The satellite is on a descending track, so moving South, and since
the satellite is East of Delft, Delft should be to the right of the
the satellite.

## Functions to compute and print lookangles

Instead of creating individual functions to compute the various
lookangles, i created one function to do all of them. This function
is called `satlookanglesp`. It uses `prtlookangle` to print a nice table.

To reproduce the results of the previous sections, with a bit more
epochs, we get in the ECI reference frame 

In [ ]:
t = tle.tledatenum(['2017-9-28 6:00',10,1])

xsat, vsat = tle.tle2vec(tleERS, t, 'RADARSAT-2')
xobj, vobj = crs.ecef2eci(t, xobjECEF)

lookangles, flags = crs.satlookanglesp(t, np.hstack([xsat, vsat]), np.hstack([xobj, vobj]))
crs.prtlookangle(t, lookangles, flags)


For the ECEF reference frame we get the same result, except for the satellite 
heading, and therefore also the look direction with respect to the flight 
direction. The heading and look angle with respect to the flight direction
are not the same in the ECI and ECEF reference frames, this is a
consequence of the fact that the ECEF reference frame is rotating.

In [ ]:
xsate, vsate = crs.eci2ecef(t, xsat, vsat)

xobje = np.array(xobjECEF)
xobje=np.tile(xobje,[t.shape[0],1])
xobje=np.hstack([xobje, np.zeros((t.shape[0],3))])

lookangles, flags = crs.satlookanglesp(t, np.hstack([xsate, vsate]),xobje)
crs.prtlookangle(t, lookangles, flags)

The difference in heading is roughly two degrees in this case. 

Note that this function expects the satellite statevector instead of
the separate position and velocities. For the object coordinates *xobje*
only the position is enough in an ECEF frame, but for an ECI frame both
position and velocity need to be given for every epoch.

For more information on how to use this function type 
```
help(satlookanglesp)
```

In [ ]:
help(crs.satlookanglesp)

## Final remarks

You may have noticed that we used astronomical latitude and longitude
to compute the position of an observer on Earth. It is also possible
to use geodetic latitude and longitude, which relate to an ellipsoid
instead of a sphere, and compute zenith angles and azimuth with respect
to the ellipsoid. These computations are supported by the `crsutil` 
module, but far more complicated than the ones shown in this example,
while the differences are very small (few tenths of a degree).

The computations are done in an ECI reference frame. In exercise 4 we 
discuss the transformation to an ECEF reference frame and redo some of
the earlier computations.

In this exercise we computed the look angles between two points, a 
satellite and an observer. Note that the observer can be a point on
Earth or another satellite. This does not really matter. In exercise 5 
we tackle the inverse problem: given the position of a satellite,
and the look angles from the satellite, find the corresponding point on 
Earth.

[End of this Jupyter notebook]